In [ ]:
#!pip install -U transformers

In [ ]:
#For offloading models to GPU.
#!pip install -q accelerate transformers

In [ ]:
#Importing all required packages
import pickle
import json
from tqdm import tqdm
import time
import os
from huggingface_hub import login
from langchain.schema import Document
import torch

# Use a pipeline as a high-level helper
from transformers import pipeline

#Cuda memory will faill without this
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

#Importing secretkey saved in KAggle secrets for logging into huggingface
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

#Logging into huggingface liek this as CLI is not working for us in Kaggle
login(token=secret_value_0)

#Importing packages required to download model
import requests
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

#For offloading model to GPU
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig

In [3]:
# #Cuda will report exact line of error
# import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["PYTORCH_USE_CUDA_DSA"] = "1"

In [2]:
with open("/kaggle/input/all-marketing-material/all_marketing_material.pkl", "rb") as f:
    all_marketing_pages = pickle.load(f)

In [3]:
len(all_marketing_pages)

6608

In [4]:
model_id = "google/gemma-3-4b-it"

#Downloading models locally to query them
model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",  # Offloads intelligently between GPU & CPU
    offload_folder="offload_dir"  # Offload excess weights to disk (temporary)
)
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
# #Downloading models locally to query them
# model = Gemma3ForConditionalGeneration.from_pretrained("google/gemma-3-4b-it")
# processor = AutoProcessor.from_pretrained("google/gemma-3-4b-it")

In [5]:
#Testing, its working when Models are in GPU
messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": "You are a helpful assistant."}
        ]
    },
    {
        "role": "user", "content": [
            {"type": "text", "text": "What is machine learning?"},
        ]
    },
]

inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
    add_generation_prompt=True
)
# Generate
generate_ids = model.generate(**inputs)
processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2347: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


"user\nYou are a helpful assistant.\n\nWhat is machine learning?\nmodel\nOkay, let's break down what machine learning is! Here's a breakdown in a way"

In [5]:
# Models on GPU but runs queries on CPU!
# def local_llm(prompt: str, processor, model) -> str:
#     """
#     Sends a prompt to a HuggingFace Gemma model and returns the response.
#     """
#     messages = [
#         {
#             "role": "system",
#             "content": [
#                 {"type": "text", "text": "You are a helpful assistant."}
#             ]
#         },
#         {
#             "role": "user",
#             "content": [
#                 {"type": "text", "text": prompt}
#             ]
#         }
#     ]

#     try:
#         inputs = processor.apply_chat_template(
#             messages,
#             tokenize=True,
#             return_dict=True,
#             return_tensors="pt",
#             add_generation_prompt=True
#         ).to(model.device)

#         with torch.inference_mode():
#             outputs = model.generate(**inputs, max_new_tokens=1024)
        
#         # Decode only the new tokens (not input prompt)
#         decoded_output = processor.batch_decode(
#             outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True
#         )[0]

#         return decoded_output.strip()
    
#     except Exception as e:
#         print(f"Error in HuggingFace LLM call: {e}")
#         return ""


In [12]:
# # This is one more working piece fo code, thats running on GPU
# messages = [
#     {
#         "role": "system",
#         "content": [{"type": "text", "text": "You are a helpful assistant."}]
#     },
#     {
#         "role": "user",
#         "content": [{"type": "text", "text": "Explain what marketing funnels are."}]
#     }
# ]

# # Step 3: Prepare inputs (tokenized + moved to model's device)
# inputs = processor.apply_chat_template(
#     messages,
#     tokenize=True,
#     return_dict=True,
#     return_tensors="pt",
#     add_generation_prompt=True
# )

# # 🧠 Step 4: Move inputs to the **same device** as model
# device = model.device  # Auto-detected GPU device
# inputs = {k: v.to(device) for k, v in inputs.items()}

# # Step 5: Generate response
# with torch.inference_mode():
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=512
#     )

# # Step 6: Decode response
# decoded_output = processor.batch_decode(
#     outputs,
#     skip_special_tokens=True,
#     clean_up_tokenization_spaces=True
# )[0]

# print(decoded_output)

user
You are a helpful assistant.

Explain what marketing funnels are.
model
Okay, let's break down marketing funnels! They're a really valuable tool for understanding and improving how you attract and convert customers. Here’s a clear explanation:

**What is a Marketing Funnel?**

A marketing funnel is a visual representation of the customer journey – the steps a potential customer takes from first becoming aware of your brand to ultimately becoming a paying customer. It’s called a “funnel” because it naturally narrows down as people move through the stages.  Lots of people enter at the top, but fewer and fewer make it all the way to the bottom.

**The Classic Stages of a Marketing Funnel:**

While variations exist, the most common model includes these stages:

1. **Awareness (Top of the Funnel - TOFU):**
   * **What it is:** This is where potential customers first *hear* about you. They might see your social media post, read a blog article, hear an ad, or stumble upon your website.
 

In [13]:
# Both on cuda 0 means both are on GPU
print("Model is on:", model.device)
print("Inputs are on:", inputs["input_ids"].device)

Model is on: cuda:0
Inputs are on: cuda:0


In [14]:
#Lets test if this works on GPU
def local_llm(prompt: str, processor, model) -> str:
    """
    Sends a prompt to a HuggingFace Gemma model and returns the response, using GPU if available.
    """
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."}]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}]
        }
    ]

    try:
        # Step 1: Tokenize the messages
        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
            add_generation_prompt=True
        )

        # Step 2: Move inputs to the model's device (GPU or CPU)
        device = model.device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Step 3: Generate output with inference mode
        with torch.inference_mode():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,
                do_sample=True,
                temperature=0.7
            )

        # Step 4: Decode generated output
        decoded_output = processor.batch_decode(
            outputs,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )[0]

        return decoded_output.strip()

    except Exception as e:
        print(f"Error in HuggingFace LLM call: {e}")
        return ""

In [15]:
def process_chunk_to_alpaca(doc: Document, processor, model) -> dict:
    source_name = doc.metadata.get("source", "Unknown Name")

    instruction_with_metadata = f"""
You are a business assistant analyzing raw business content from the following source:
SOURCE NAME: {source_name}

Your task is to extract the following from the provided transcript:
1. Frameworks (e.g., naming, advertising, validation models).
2. Bullet points for key ideas or steps.
3. Q&A (any implied or stated questions with answers).
4. Case Examples or stories.
5. Copywriting formulas (AIDA, PAS, etc.)
6. Classify this content into high-level topics: e.g., Naming, Ads, Psychology, Copywriting.
7. Convert suitable content into a step-by-step guide.

Return your output in clearly labeled sections, and only include sections with relevant content. Do not include a preamble.
""".strip()

    prompt = f"{instruction_with_metadata}\n\n{doc.page_content.strip()}"
    response = local_llm(prompt, processor, model)

    return {
        "instruction": instruction_with_metadata,
        "input": doc.page_content.strip(),
        "output": response,
        "metadata": doc.metadata
    }


In [16]:
alpaca_data = []

for doc in all_marketing_pages[:1]:  # or full range
    alpaca_entry = process_chunk_to_alpaca(doc, processor, model)
    alpaca_data.append(alpaca_entry)

In [17]:
alpaca_data

[{'instruction': 'You are a business assistant analyzing raw business content from the following source:\nSOURCE NAME: Alex Hormozi 100 million leads\n\nYour task is to extract the following from the provided transcript:\n1. Frameworks (e.g., naming, advertising, validation models).\n2. Bullet points for key ideas or steps.\n3. Q&A (any implied or stated questions with answers).\n4. Case Examples or stories.\n5. Copywriting formulas (AIDA, PAS, etc.)\n6. Classify this content into high-level topics: e.g., Naming, Ads, Psychology, Copywriting.\n7. Convert suitable content into a step-by-step guide.\n\nReturn your output in clearly labeled sections, and only include sections with relevant content. Do not include a preamble.',
  'input': 'A c q u i s i t i o n . c o m  V o l u m e  I I\n \n$100M Leads\n \nH o w  t o  G e t  S t r a n g e r s  T o\nW a n t  T o  B u y  Y o u r  S t u f f\n \n \nA l e x  H o r m o z i\nCopyright © 2023 by Alex Hormozi\n \nAll rights reserved. No part of thi

In [ ]:
# Lets make a state managed code that processes all docs with tqdm

In [32]:
PROCESSED_FILE = "alpaca_processed.jsonl"
FAILED_FILE = "alpaca_failed.jsonl"
MAX_RETRIES = 3
RETRY_DELAY = 2  # seconds between retries

def load_jsonl_ids(filename):
    if not os.path.exists(filename):
        return set()
    with open(filename, "r") as f:
        return {json.loads(line).get("metadata", {}).get("source", "") + str(json.loads(line).get("metadata", {}).get("page", "")) for line in f}

def save_jsonl(filename, data):
    with open(filename, "a") as f:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

def get_doc_id(doc: Document):
    source = doc.metadata.get("source", "")
    page = doc.metadata.get("page")

    if page is None:
        # Fallback to hashing part of the content if page is missing
        content_hash = str(abs(hash(doc.page_content[:50])))
        return f"{source}_hash_{content_hash}"

    return f"{source}_page_{page}"

def process_documents_with_retries(pages, processor, model):
    processed_ids = load_jsonl_ids(PROCESSED_FILE)
    failed_ids = load_jsonl_ids(FAILED_FILE)

    for doc in tqdm(pages, desc="Processing documents"):
        doc_id = get_doc_id(doc)

        if doc_id in processed_ids:
            continue

        retries = 0
        success = False

        while retries < MAX_RETRIES and not success:
            try:
                alpaca_entry = process_chunk_to_alpaca(doc, processor, model)
                save_jsonl(PROCESSED_FILE, alpaca_entry)
                success = True
            except Exception as e:
                retries += 1
                if retries < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    error_entry = {
                        "error": str(e),
                        "metadata": doc.metadata,
                        "input": doc.page_content[:500]  # preview of failed input
                    }
                    save_jsonl(FAILED_FILE, error_entry)
        time.sleep(1)

In [ ]:
process_documents_with_retries(all_marketing_pages, processor, model)

Processing documents:   0%|          | 7/6608 [07:03<108:25:02, 59.13s/it]